## 📌 Théorie : Modèle Hydraulique

### 👉 Objectif : Calculer la hauteur d’eau  H  et la surverse  S  en fonction du débit  Q .

On utilise une équation hydraulique simplifiée pour estimer la hauteur d’eau $H$ en fonction du débit $Q$ :

$$H = \left( \frac{Q}{K_s \sqrt{\frac{Z_m - Z_v}{L} B}} \right)^{\frac{3}{5}}$$

où :
- $Q$ : débit de la rivière ($\text{m}^3$/s)
- $K_s$ : coefficient de Strickler
- $Z_m, Z_v$ : cotes du fond de la rivière en amont et aval
- $L$ : longueur du tronçon étudié (m)
- $B$ : largeur du cours d’eau (m)

📌 **Surverse $S$** : dépassement de la digue

$$S = Z_v + H - h_d - Z_b$$

où :
- $h_d$ : hauteur de la digue
- $Z_b$ : cote de la berge

### 📂 Importation des données & librairies

In [1]:
from utils import *
import numpy as np
import plotly.express as px
params = load_hydraulic_params()

In [2]:
df = load_cleaned_data()

In [3]:
df.head()

,Annee,Debit,Hauteur
0,1850,1256.402143,4.0
1,1851,1648.542191,4.5
2,1852,1604.857094,4.3
3,1853,341.157419,1.7
4,1854,1148.712945,3.4



| Paramètre dans le code | Notation mathématique | Explication |
|------------------------|----------------------|-------------|
| `params["L"]`  | $L$  | Longueur du tronçon étudié (m) |
| `params["B"]`  | $B$  | Largeur moyenne de la rivière (m) |
| `params["Ks"]` | $K_s$ | Coefficient de Strickler (rugosité du lit) |
| `params["Zm"]` | $Z_m$ | Cote du fond en amont (m NGF) |
| `params["Zv"]` | $Z_v$ | Cote du fond en aval (m NGF) |
| `params["Zb"]` | $Z_b$ | Cote de la berge (m NGF) |
| `params["hd"]` | $h_d$ | Hauteur de la digue estimée (m) |

In [4]:
L = params["L"]
B = params["B"]
Ks = params["Ks"]
Zm = params["Zm"]
Zv = params["Zv"]
Zb = params["Zb"]
hd = params["hd"]
print("Paramètres du modèle hydraulique :")
print(params)

Paramètres du modèle hydraulique :
{'L': 5000, 'B': 300, 'Ks': 30, 'Zm': 55, 'Zv': 50, 'Zb': 55.5, 'hd': 5.5, 'H_IC_95': [1.19, 4.03], 'S_IC_95': [-10.72, -7.36]}


$$H = \left( \frac{Q}{K_s \sqrt{\frac{Z_m - Z_v}{L} B}} \right)^{\frac{3}{5}}$$

In [5]:
df["H_calculé"] = (df["Debit"] / (Ks * np.sqrt((Zm - Zv) / L) * B)) ** (3/5)
df[["Debit", "H_calculé"]].head()

,Debit,H_calculé
0,1256.402143,2.437427
1,1648.542191,2.868894
2,1604.857094,2.823035
3,341.157419,1.114878
4,1148.712945,2.309837


$$S = Z_v + H - h_d - Z_b$$


In [6]:
df["S_calculé"] = Zv + df["H_calculé"] - hd - Zb
df[["Debit", "H_calculé", "S_calculé"]].head()

,Debit,H_calculé,S_calculé
0,1256.402143,2.437427,-8.562573
1,1648.542191,2.868894,-8.131106
2,1604.857094,2.823035,-8.176965
3,341.157419,1.114878,-9.885122
4,1148.712945,2.309837,-8.690163


In [7]:
fig_H = px.scatter(df, x="Debit", y="H_calculé", title="Relation entre le débit et la hauteur d'eau",
                   labels={"Debit": "Débit (m³/s)", "H_calculé": "Hauteur d'eau (m)"})
fig_H.show()

fig_S = px.scatter(df, x="Debit", y="S_calculé", title="Relation entre le débit et la surverse",
                   labels={"Debit": "Débit (m³/s)", "S_calculé": "Surverse (m)"})
fig_S.show()

> 	La surverse  S  est toujours négative, ce qui signifie que l’eau ne dépasse jamais la digue

💡 Les résultats montrent que la digue est bien dimensionnée pour les crues observées dans les données historiques. Toutefois, une analyse économique est nécessaire pour voir si cette hauteur de digue est optimale en termes de coûts de construction et de maintenance.